In [2]:
import pandas as pd
import pandas_ta as ta

# Get Data

In [3]:
ohlc_df = pd.read_csv("ETHUSDT-1h-2022-11.csv")
ohlc_df["timestamp"] = pd.to_datetime(ohlc_df.timestamp, unit="ms")

In [4]:
ohlc_df["rsi"] = ta.rsi(ohlc_df.close)
ohlc_df = ohlc_df.set_index("timestamp")
ohlc_df

,open,high,low,close,volume,rsi
timestamp,,,,,,
2022-11-01 00:00:00,1572.69,1576.46,1568.80,1570.11,21181.2714,NaN
2022-11-01 01:00:00,1570.11,1584.37,1569.68,1581.18,18171.1009,NaN
2022-11-01 02:00:00,1581.18,1600.46,1578.62,1598.35,29111.9684,NaN
2022-11-01 03:00:00,1598.35,1604.73,1583.94,1586.81,19500.0908,NaN
2022-11-01 04:00:00,1586.81,1593.67,1580.32,1589.63,16578.1697,NaN
...,...,...,...,...,...,...
2022-11-30 19:00:00,1278.26,1300.00,1278.01,1287.20,73361.3997,70.887534
2022-11-30 20:00:00,1287.20,1295.76,1285.00,1295.29,20934.6132,73.711896
2022-11-30 21:00:00,1295.29,1299.00,1287.73,1297.07,23886.5728,74.302623


# Generate Trades

In [5]:
rsi_upper_threshold = 70
rsi_lower_threshold = 30

trade_value = 100
trades = []

current_trade = {}

for i in range(len(ohlc_df) -1):
    
    if ohlc_df.iloc[i].rsi > rsi_upper_threshold and len(current_trade) != 0:
        
        trades.append({
            "entry_price":current_trade["entry_price"],
            "entry_time":current_trade["entry_time"],
            "trade_size":current_trade["remaining_size"],
            "exit_price":ohlc_df.iloc[i+1].open,
            "exit_time":ohlc_df.iloc[i+1].name,
            "profit_pct":(ohlc_df.iloc[i+1].open/current_trade["entry_price"]) -1,
        })
        
        current_trade = {}
        
    elif ohlc_df.iloc[i].rsi < rsi_lower_threshold and len(current_trade) == 0:
        
        current_trade["entry_price"] = ohlc_df.iloc[i+1].open
        current_trade["entry_time"] = ohlc_df.iloc[i+1].name
        current_trade["initial_size"] = trade_value / current_trade["entry_price"]
        current_trade["remaining_size"] = current_trade["initial_size"]

trades = pd.DataFrame(trades)
trades

,entry_price,entry_time,trade_size,exit_price,exit_time,profit_pct
0,1511.27,2022-11-02 21:00:00,0.066170,1587.43,2022-11-04 08:00:00,0.050395
1,1568.25,2022-11-07 00:00:00,0.063765,1224.20,2022-11-19 23:00:00,-0.219385
2,1167.76,2022-11-20 12:00:00,0.085634,1214.00,2022-11-24 07:00:00,0.039597
3,1164.00,2022-11-28 02:00:00,0.085911,1215.06,2022-11-29 09:00:00,0.043866


# Plotting

In [31]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import timedelta

fig = make_subplots(rows=2, cols = 1, shared_xaxes=True, row_heights = [0.75, 0.25],
                    vertical_spacing=0.1)

fig.add_trace(go.Candlestick(
                x = ohlc_df.index,
                open = ohlc_df.open,
                high = ohlc_df.high,
                low = ohlc_df.low,
                close = ohlc_df.close, 
                increasing_line_color = 'rgba(107,107,107,0.8)',
                decreasing_line_color = 'rgba(210,210,210,0.8)',
                name = "OHLC"),
             row=1, col=1)


fig.add_trace(go.Scatter(
                x=trades.entry_time,
                y=trades.entry_price,
                mode = "markers",
                customdata=trades,
                marker_symbol="diamond-dot",
                marker_size = 13,
                marker_line_width = 2,
                marker_line_color = "rgba(0,0,0,0.7)",
                marker_color="rgba(0,255,0,0.7)",
                hovertemplate="Entry Time: %{customdata[1]}<br>" +\
                              "Entry Price: %{y:.2f}<br>" +\
                              "Size: %{customdata[2]:.5f}<br>" +\
                              "Profit_pct: %{customdata[5]:.3f}",
                name="Entries"),
             row=1, col=1)

fig.add_trace(go.Scatter(
                x=trades.exit_time,
                y=trades.exit_price,
                mode = "markers",
                customdata=trades,
                marker_symbol="diamond-dot",
                marker_size = 13,
                marker_line_width = 2,
                marker_line_color = "rgba(0,0,0,0.7)",
                marker_color="rgba(255,0,0,0.7)",
                hovertemplate="Exit Time: %{customdata[4]}<br>" +\
                              "Exit Price: %{y:.2f}<br>" +\
                              "Size: %{customdata[2]:.5f}<br>" +\
                              "Profit_pct: %{customdata[5]:.3f}",
                name="Exits"),
             row=1, col=1)

fig.add_trace(
        go.Scatter(x=ohlc_df.index, y=ohlc_df.rsi, name="RSI"),
    row=2,col=1)

bar_freq = timedelta(hours=1)

fig.add_trace(go.Scatter(
                x=trades.entry_time - bar_freq,
                y=ohlc_df.rsi.loc[trades.entry_time - bar_freq],
                mode = "markers",
                customdata=trades,
                marker_symbol="circle-dot",
                marker_size = 8,
                marker_line_width = 1,
                marker_line_color = "rgba(0,0,0,0.7)",
                marker_color="rgba(0,255,0,0.7)",
                name="Indicator Entry"),
             row=2, col=1)

fig.add_trace(go.Scatter(
                x=trades.exit_time - bar_freq,
                y=ohlc_df.rsi.loc[trades.exit_time - bar_freq],
                mode = "markers",
                customdata=trades,
                marker_symbol="circle-dot",
                marker_size = 8,
                marker_line_width = 1,
                marker_line_color = "rgba(0,0,0,0.7)",
                marker_color="rgba(255,0,0,0.7)",
                name="Indicator Exits"),
             row=2, col=1)

fig.update_layout(xaxis_rangeslider_visible=False)

fig